Load API Keys

In [ ]:
from dotenv import load_dotenv
load_dotenv()

Load Contents from HLB Fixed Deposite Pages

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

Split the Loaded Data into Chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(data)
texts

Initialize Pinecone

In [ ]:
import os
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "main-index"
namespace = "hlb-fixed-deposit-with-conversation-chain-gemini"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

Create Embeddings and Vectors to Insert to Pinecone Database

In [ ]:
from langchain_community.vectorstores.pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = Pinecone.from_documents(texts, embeddings, index_name=index_name, namespace=namespace)

Create LLM

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.1, convert_system_message_to_human=True  )

Creating Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_template = """Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
If there is no context, don't answer with your own knowledge, just say you don't know.
----------------
{context}
"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]
prompt = ChatPromptTemplate.from_messages(messages)

Create Retriever for Vector Search

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

Compressor Pipeline

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

embedding_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.8)

Contextual Compression

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever

compression_retriever = ContextualCompressionRetriever(
    base_compressor=embedding_filter, base_retriever=retriever
)

Creating Memory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(llm=llm, memory_key="chat_history", output_key="answer", k=2, return_messages=True)

Creating Chain

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain_community.callbacks.manager import get_openai_callback
 
chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type="stuff",
  retriever=compression_retriever,
  memory=memory,
  combine_docs_chain_kwargs={"prompt": prompt},
  verbose=True,
  return_source_documents=True,
  max_tokens_limit=3000
)

def ask_question(question):
  with get_openai_callback() as cb:
    result = chain.invoke(question)
    for source_documents in result["source_documents"]:
      print(f'\n{source_documents}')
    print(f'\nDocument: {len(result["source_documents"])}')
    print(f'\nAnswer: {result["answer"]}')
    print(f'{cb}\n')

Running the Chain

In [ ]:
ask_question("What is fixed deposit?")

In [ ]:
ask_question("How many types of fixed deposit does HongLeong Bank provide?")

In [ ]:
ask_question("What is the difference between Fixed Deposit and eFixed Deposit?")

In [ ]:
ask_question("What are the interest rates for Fixed Deposit?")

In [ ]:
ask_question("Let's say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.")